# Imports and Environment Configuration

In [1]:
import os
import gc
import cv2
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.cuda import amp
import torchvision
import timm
import albumentations as A
from albumentations.pytorch import ToTensorV2
import joblib
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
import warnings

warnings.filterwarnings("ignore")
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Configuration 

In [2]:
CONFIG = {
    "seed": 42,
    "img_size": 512,  #  experiment with this for performance vs accuracy trade-off
    "model_name": "tf_efficientnetv2_s_in21ft1k",
    "num_classes": 6,  # Including 'Other' class
    "valid_batch_size": 32,
    "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
}

# Seed setting function

In [3]:
def set_seed(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)

set_seed(CONFIG['seed'])

# Data Paths

In [4]:
ROOT_DIR = '/kaggle/input/UBC-OCEAN'
TEST_DIR = '/kaggle/input/UBC-OCEAN/test_thumbnails'
ALT_TEST_DIR = '/kaggle/input/UBC-OCEAN/test_images'
LABEL_ENCODER_BIN = "/kaggle/input/ubcpytorchwith-classweights-training-fold1of5/label_encoder.pkl"
BEST_WEIGHT = "/kaggle/input/baseline-0-36/Acc0.70_Loss1.0140_epoch29_tf_efficientnetv2_s_in21ft1k_0.36.bin"

# Test File Path Function

In [5]:
def get_test_file_path(image_id):
    thumbnail_path = f"{TEST_DIR}/{image_id}_thumbnail.png"
    alt_path = f"{ALT_TEST_DIR}/{image_id}.png"
    return thumbnail_path if os.path.exists(thumbnail_path) else alt_path

# Data Loading

In [6]:
df = pd.read_csv(f"{ROOT_DIR}/test.csv")
df['file_path'] = df['image_id'].apply(get_test_file_path)
df['label'] = 0  # Dummy value for consistency


# Sample Submission

In [7]:
df_sub = pd.read_csv(f"{ROOT_DIR}/sample_submission.csv")

# Encoder

In [8]:
encoder = joblib.load(LABEL_ENCODER_BIN)

# Dataset Class

In [9]:
class UBCDataset(Dataset):
    def __init__(self, df, transforms=None):
        self.df = df
        self.file_names = df['file_path'].values
        self.labels = df['label'].values  # Not used in test, but kept for consistency
        self.transforms = transforms
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        img_path = self.file_names[index]
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        if self.transforms:
            img = self.transforms(image=img)["image"]
            
        return {
            'image': img,
            'label': torch.tensor(self.labels[index], dtype=torch.long)
        }

# Data Transformations

In [10]:
data_transforms = {
    "valid": A.Compose([
        A.Resize(CONFIG['img_size'], CONFIG['img_size']),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
        ToTensorV2()], p=1.)
}

# Model Definition

In [11]:
class GeM(nn.Module):
    def __init__(self, p=3, eps=1e-6):
        super(GeM, self).__init__()
        self.p = nn.Parameter(torch.ones(1) * p)
        self.eps = eps

    def forward(self, x):
        return F.avg_pool2d(x.clamp(min=self.eps).pow(self.p), (x.size(-2), x.size(-1))).pow(1. / self.p)

class UBCModel(nn.Module):
    def __init__(self, model_name, num_classes, pretrained=False):
        super(UBCModel, self).__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained)
        in_features = self.model.classifier.in_features
        self.model.classifier = nn.Identity()
        self.model.global_pool = nn.Identity()
        self.pooling = GeM()
        self.linear = nn.Linear(in_features, num_classes)  # Renamed to 'linear'

    def forward(self, images):
        features = self.model(images)
        pooled_features = self.pooling(features).flatten(1)
        output = self.linear(pooled_features)  # Use self.linear here
        return output

model = UBCModel(CONFIG['model_name'], CONFIG['num_classes'], pretrained=False)  # Initialize without pretrained weights

# Load the state dict of the pretrained model except for the final layer
pretrained_dict = torch.load(BEST_WEIGHT)
model_dict = model.state_dict()

# Filter out the linear layer from pretrained dict
pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict and 'linear' not in k}

# Update current model's dict with pretrained dict and load it
model_dict.update(pretrained_dict)
model.load_state_dict(model_dict)

model.to(CONFIG['device'])

UBCModel(
  (model): EfficientNet(
    (conv_stem): Conv2dSame(3, 24, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn1): BatchNormAct2d(
      24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): SiLU(inplace=True)
    )
    (blocks): Sequential(
      (0): Sequential(
        (0): ConvBnAct(
          (conv): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNormAct2d(
            24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): SiLU(inplace=True)
          )
          (drop_path): Identity()
        )
        (1): ConvBnAct(
          (conv): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNormAct2d(
            24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): SiLU(inplace=True)
          )
 

# DataLoader Initialization

In [12]:
test_dataset = UBCDataset(df, transforms=data_transforms["valid"])
test_loader = DataLoader(test_dataset, batch_size=CONFIG['valid_batch_size'], num_workers=2, shuffle=False, pin_memory=True)

# Prediction Loop

In [13]:
preds = []
model.eval()
with torch.no_grad():
    for data in tqdm(test_loader):
        images = data['image'].to(CONFIG["device"], dtype=torch.float)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        preds.extend(predicted.detach().cpu().numpy())

pred_labels = encoder.inverse_transform(preds)

100%|██████████| 1/1 [00:04<00:00,  4.93s/it]


# Create Submission File

In [14]:
df_sub["label"] = pred_labels
df_sub.to_csv("submission.csv", index=False)

In [15]:
df_sub

,image_id,label
0,41,HGSC
